In [172]:
from bs4 import BeautifulSoup as b
import requests as rqt
import pandas as pd
import time

In [173]:
URL = "https://www.pro-football-reference.com/draft/2024-combine.htm"

r = rqt.get(URL)

if r.status_code == 429:
    retry_after = r.headers.get('Retry-After')
    if retry_after:
        wait_time = int(retry_after)
        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
        time.sleep(wait_time)

In [174]:
soup = b(r.text, "lxml")

table_head = soup.find("thead")
headers = table_head.find_all("th")

column_names = []
for i in headers:
    value = i.text
    column_names.append(value)

print(column_names)

['Player', 'Pos', 'School', 'College', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted (tm/rnd/yr)']


In [175]:
df = pd.DataFrame(columns=['Player', 'Pos', 'stats_link', 'Ht', 'Wt', '40yd', 'Vertical', 'Bench', 'Broad Jump', '3Cone', 'Shuttle', 'Drafted (tm/rnd/yr), year'])

In [176]:
table_body = soup.find("tbody")
body = table_body.find_all("tr")
Player_details = []

for i in body:
    th = i.find("th")

    if th:
        a = th.find("a")
        if a:
            name = a.text
            if name == 'Player':
                continue
            
            name_link = a["href"]
            '''if name_link:
                print("https://www.pro-football-reference.com" + str(name_link))
                P_Req = rqt.get("https://www.pro-football-reference.com" + str(name_link))
                if P_Req.status_code == 429:
                    retry_after = P_Req.headers.get('Retry-After')
                    if retry_after:
                        wait_time = int(retry_after)
                        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
                        time.sleep(wait_time)
                P_soup = b(P_Req.text, "lxml")
                DOB_ele = P_soup.find(id="necro-birth")
                print(DOB_ele)
                if DOB_ele.text:
                    DOB = DOB_ele.text.split(',')[1][1 : 5]
                    print(DOB)
                else:
                    DOB = " "
                    print(DOB)


            else:
                name_link = "https://www.pro-football-reference.com/players/" + name[name.find(" ")+1] + "/" + name[name.find(" ")+1 : name.find(" ")+5] + name[0:2] + "00.htm"
                print(name_link)
                P_Req = rqt.get(name_link)
                if P_Req.status_code == 429:
                    retry_after = P_Req.headers.get('Retry-After')
                    if retry_after:
                        wait_time = int(retry_after)
                        print(f"Rate limit exceeded. Retrying after {wait_time} seconds.")
                        time.sleep(wait_time)
                P_soup = b(P_Req.text, "lxml")
                print(P_soup)
                DOB_ele = P_soup.find(id="necro-birth")
                print(DOB_ele)
                if DOB_ele.text:
                    DOB = DOB_ele.text.split(',')[1][1 : 5]
                else:
                    DOB = " "'''

                                

    td = i.find_all("td")
    for i in td:
        if i["data-stat"] == "pos":
            pos = i.text

        if i["data-stat"] == "college":
            stats = i.find("a")
            if stats:
                stats_link = stats["href"]
            else:
                stats_link = " "

        if i["data-stat"] == "height":
            height = i.text

        if i["data-stat"] == "weight":
            weight = i.text

        if i["data-stat"] == "forty_yd":
            forty_yd = i.text

        if i["data-stat"] == "vertical":
            vertical = i.text

        if i["data-stat"] == "bench_reps":
            bench_reps = i.text

        if i["data-stat"] == "broad_jump":
            broad_jump = i.text
        
        if i["data-stat"] == "cone":
            cone = i.text

        if i["data-stat"] == "shuttle":
            shuttle = i.text

        if i["data-stat"] == "draft_info":
            draft_info = i.text

    row = {'Player': name, 'Pos': pos, 'stats_link': stats_link, 'Ht': height, 'Wt': weight, '40yd': forty_yd, 'Vertical': vertical, 'Bench': bench_reps, 'Broad Jump': broad_jump, '3Cone': cone, 'Shuttle': shuttle, 'Drafted (tm/rnd/yr)': draft_info, 'year': 2024}

    Player_details.append(row)

df = pd.DataFrame(Player_details)




In [177]:
print(df)

                 Player Pos  \
0    Kris Abrams-Draine  CB   
1          Isaiah Adams   G   
2           Rasheen Ali  RB   
3             Erick All  TE   
4         Braelon Allen  RB   
..                  ...  ..   
322        Roman Wilson  WR   
323         Mekhi Wingo  DT   
324       Xavier Worthy  WR   
325       Jaylen Wright  RB   
326          Zak Zinter   G   

                                            stats_link    Ht   Wt  40yd  \
0    https://www.sports-reference.com/cfb/players/k...  5-11  179  4.44   
1    https://www.sports-reference.com/cfb/players/i...   6-4  315  5.22   
2    https://www.sports-reference.com/cfb/players/r...  5-11  206         
3    https://www.sports-reference.com/cfb/players/e...   6-4  252         
4    https://www.sports-reference.com/cfb/players/b...   6-1  235         
..                                                 ...   ...  ...   ...   
322  https://www.sports-reference.com/cfb/players/r...  5-11  185  4.39   
323  https://www.sports-ref

In [178]:
df.to_csv('D:\\602_IntroToML\\NFLDraftPrediction\\Data\\Data_2024.csv', index=False)